# Proyecto II

**Curso de Inteligencia Artificial**  
**Escuela de Ingeniería en Computación**  
**Instituto Tecnológico de Costa Rica**

## I. OBJETIVO

Aplicar un experimento que permita validar la hipótesis de que al aplicar técnicas de destilado de modelos de grandes volúmenes de parámetros en modelos más pequeños se pueden resolver tareas igual de complejas pero con modelos más eficientes.

## II. MODELO DE DETECCIÓN DE ANOMALÍAS

Para el desarrollo de este proyecto debe usar el dataset propuesto en **MVTec AD — A Comprehensive Real-World Dataset for Unsupervised Anomaly Detection**. Un dataset de escenarios industriales reales con diferentes tipos de anomalías en la forma de detección de defectos en objetos o texturas.

Seleccione **10 clases del dataset** las que usted más prefiera, y con este subconjunto vamos a entrenar distintos modelos para resolver un problema de detección de anomalías.

III. MODELOS
siguiendo la estructura de RESNET-18 para las primeras 3
Cada modelo debe estructurar el proyecto utilizando la
convoluciones (conv1, con2 x, con3 x) (ver Figura 1), de aca´
librer´ıa Hydra para la gestio´n modular de configuraciones,
en adelante coloque un clasificador (FC layer) a su gusto para
asegurando la correcta separacio´n de hiper para´metros entre
crear un clasificador entre las distintas clases.
el modelo, el entrenamiento y los registros experimentales.
Una vez disen˜ado el modelo debe proceder hacer dos
La estructura m´ınima recomendada del proyecto es la sigu-
variantes A y B.
iente:
ElmodeloAsera´ entrenadodesde0esdeciraliniciotendra´
conf/
pesos colocados aleatoriamente y comenzara´ su proceso de
- config.yaml
entrenamiento.
- model/
El modelo B será entrenando siguiendo un proceso de des-
- vae.yaml
tilado del modelo RESNET-18 siguiendo la te´cnica teacher-
- trainer/
student donde el modelo RESNET-18 sirve como teacher y
- default.yaml
el modelo B como student.
- logger/
Para esto debe investigar como aplicar correctamente un
- wandb.yaml
proceso de destilado de un modelo.
Cada mo´dulo de configuracio´n debera´ permitir la ejecucio´n Es importante un buen disen˜o de modelo que permita
de experimentos con distintos para´metros del modelo, tales obtener el vector de embeddings de salida de las capas
como: convolucionales. Pues son los que luego permitira´n disen˜ar
• Dimensio´n del espacio latente (z). el detector de anomal´ıas.
• Cantidad de e´pocas, taman˜o de batch, o cualquier hiper- Cada modelo debe ser entrenado al menos con 3 hiper-
parametro que requiera. pametros distintos para obtener buenos modelos y no sola-
Adema´s debe utilizar Pytorch Ligthning para las person- mente la primera combinacio´n que obtengan.
alizaciones de los entrenamientos y creacio´n de los modelos
basado en las mejores pra´cticas de disen˜o de software que
B. Modelo C - Embedding de un autoencoder
permita una correcto disen˜o escalable. Debe crear su propia
clase de carga de datos utilizando “LightningDataModule” y Disen˜o un modelo de autoencoder basado en U-net que re-
su modelo utilizando “LightningModule”, aca´ debe redefinir construya las ima´genes del set de entrenamiento seleccionado
como m´ınimo los me´todos de training step, test step, config- y tambie´n permita obtener el embedding correspondiente.
ure optimizers. Este sera´ entrenado completamente desde 0.
Fig.1. ArquitecturadelaredResNet-18.

## IV. EVALUACIÓN DE ANOMALÍAS

Una vez entrenados los modelos, se deben calcular las representaciones latentes (embeddings) de las imágenes del conjunto de validación para estimar una métrica que permita, posteriormente, identificar los datos anómalos en el conjunto de prueba.

### 1) Ejemplo: Mahalanobis Distance

Una manera estadística de determinar si una nueva muestra pertenece o no a la distribución de los datos normales consiste en utilizar la **distancia de Mahalanobis**.

Esta métrica mide la distancia entre un vector de características (embedding) y la distribución multivariada estimada a partir de las muestras normales, considerando las correlaciones entre dimensiones.

#### 1. Estimación de la distribución normal

A partir del conjunto de validación o entrenamiento correspondiente a la clase sin defectos, se extraen los embeddings de cada imagen mediante el modelo previamente entrenado (para cada modelo).

Cada embedding puede representarse como un vector $\mathbf{z}_{i} \in \mathbb{R}^{d}$. Con todos los embeddings del conjunto normal se calcula la media $\boldsymbol{\mu}$ y la matriz de covarianza $\boldsymbol{\Sigma}$:

$$
\boldsymbol{\mu}=\frac{1}{N} \sum_{i=1}^{N} \mathbf{z}_{i}, \quad \boldsymbol{\Sigma}=\frac{1}{N-1} \sum_{i=1}^{N}\left(\mathbf{z}_{i}-\boldsymbol{\mu}\right)\left(\mathbf{z}_{i}-\boldsymbol{\mu}\right)^{T}
$$

De esta forma se modela la distribución normal como una distribución gaussiana multivariada $\mathcal{N}(\boldsymbol{\mu}, \boldsymbol{\Sigma})$, que representa los datos normales en el espacio de embeddings.

#### 2. Cálculo de la distancia de Mahalanobis

Para una nueva muestra con embedding $\mathbf{z}_{\text{test}}$, se calcula su distancia a la distribución normal.

Esta distancia mide qué tan alejada se encuentra la muestra del centro de la distribución de los datos sin defectos, considerando la forma y correlaciones de dicha distribución.

A partir de acá debe de averiguar como clasificar una anomalía o una clase sin defectos utilizando comparación de la distancia (e.g tomar el percentil).

**Nota:** El estudiante puede implementar también otras estrategias de detección, como la distancia euclidiana, reconstrucción basada en error (reconstruction loss).

## V. MODELOS CUANTIZADOS

Adicionalmente se desea hacer comparaciones de los resultados entre los modelos originales y los modelos cuantizados para hacer comparaciones de rendimiento. Para esto, convierta los **tres modelos con mejores resultados** de acuerdo a su criterio a modelos cuantizados, y realice una comparación de latencias en respuesta, tamaño, y rendimiento, incluya este análisis en su informe.

## VI. ANÁLISIS DE OUTLIERS MEDIANTE DBSCAN CLUSTERING

Una vez identificado el mejor modelo de detección de anomalías —ya sea el clasificador CNN entrenado desde cero, su versión distilada mediante teacher–student, o el modelo autoencoder basado en U-Net— proceda a utilizar sus embeddings como insumo para realizar un análisis adicional mediante técnicas de agrupamiento no supervisado. En particular **DBSCAN** (Density-Based Spatial Clustering of Applications with Noise), un método basado en densidad que permite identificar regiones de alta concentración en el espacio latente y, simultáneamente, detectar puntos aislados que pueden interpretarse como outliers o anomalías.

Extraiga los embeddings generados por el modelo seleccionado para cada imagen del conjunto de prueba. En estos espacios latentes, las imágenes normales suelen agruparse de forma natural alrededor de regiones de alta densidad, mientras que las anomalías producen vectores que tienden a ubicarse lejos de dichas regiones. Con el fin de facilitar tanto la visualización como la separación estructural, aplique reducción de dimensionalidad con **PCA** y **t-SNE**.

Una vez obtenidas las representaciones latentes reducidas aplique **DBSCAN**. Desde la perspectiva de la detección de anomalías, los puntos etiquetados por DBSCAN como ruido constituyen una indicación natural de potencial anomalía, ya que representan vectores que se encuentran en zonas de baja densidad del espacio latente.

Analice los resultados desde el punto de vista visual, y cuantitativa del resultado de la clasificación de anomalías.

---

## RÚBRICA

Si el trabajo no se encuentra debidamente ordenado y presentado siguiendo una adecuada estructura para el informe, puede ser considerado como incompleto y cualquiera de las rúbricas se puede ver afectada (ver Tabla I).

### TABLA I - RÚBRICA

| Criterio | Puntaje Máx. |
| :-- | :--: |
| Implementación de Modelo CNN Scratch y Destilado (Pytorch Lightning, Hydra y WandB) | 15 |
| Implementación de Autoencoder U-Net (Pytorch Lightning, Hydra y WandB) | 15 |
| Diseño experimental, múltiples entrenamientos y variación de hiperparámetros. | 10 |
| Comparación de modelos base: reconstrucción de imágenes, progreso de validación y entrenamiento, análisis de overfitting | 10 |
| Definición de evaluación de anomalías con embeddings | 10 |
| Comparación de mejores modelos de detección de anomalías | 10 |
| Comparación entre modelos originales y cuantizados (latencia, tamaño, rendimiento) | 10 |
| Comparación de análisis de anomalías con DBSCAN: t-SNE y PCA | 15 |
| Calidad de informe científico | 10 |
| **Total** | **105** |